In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("search_filter").getOrCreate()
cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print("You are working with {} cores".format(cores))

spark

You are working with 1 cores


In [5]:
#getting all the dataset
import os

path="../../data/"

for filename in os.listdir(path):
    if filename.endswith('.csv'):
        splitter = filename.split('.')
        df_name = splitter[0]
        df = spark.read.csv(path+filename,inferSchema=True,header=True)
        exec(df_name + "= df")
        

In [6]:
fifa19.limit(3).toPandas()

,_c0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,...,96,33,28,26,6,11,15,14,8,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,...,95,28,31,23,7,11,15,14,11,€127.1M
2,2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,...,94,27,24,33,9,9,15,15,11,€228.1M


In [8]:
fifa = fifa19

#### Let's check out schema of fifa dataframe

In [11]:
fifa.printSchema()




root
 |-- _c0: integer (nullable = true)
 |-- ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Photo: string (nullable = true)
 |-- Nationality: string (nullable = true)
 |-- Flag: string (nullable = true)
 |-- Overall: integer (nullable = true)
 |-- Potential: integer (nullable = true)
 |-- Club: string (nullable = true)
 |-- Club Logo: string (nullable = true)
 |-- Value: string (nullable = true)
 |-- Wage: string (nullable = true)
 |-- Special: integer (nullable = true)
 |-- Preferred Foot: string (nullable = true)
 |-- International Reputation: integer (nullable = true)
 |-- Weak Foot: integer (nullable = true)
 |-- Skill Moves: integer (nullable = true)
 |-- Work Rate: string (nullable = true)
 |-- Body Type: string (nullable = true)
 |-- Real Face: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- Jersey Number: integer (nullable = true)
 |-- Joined: string (nullable = true)
 |-- Loaned From: string (nu

In [10]:
fifa.select(["Name","Position"]).show(5,False)

+-----------------+--------+
|Name             |Position|
+-----------------+--------+
|L. Messi         |RF      |
|Cristiano Ronaldo|ST      |
|Neymar Jr        |LW      |
|De Gea           |GK      |
|K. De Bruyne     |RCM     |
+-----------------+--------+
only showing top 5 rows



In [ ]:
#overall이 가장 높은 선수 5명을 출력해보자. 
#Potential이 가장 높은선수 5명도 해보자. 
#Release Clause를 Float형태로 변환해서 상관관계 분석도 ㄱㄱ

In [15]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

#1-1
fifa.select(["Name","overall"]).orderBy(desc("overall")).limit(5).toPandas()

,Name,overall
0,L. Messi,94
1,Cristiano Ronaldo,94
2,Neymar Jr,92
3,K. De Bruyne,91
4,De Gea,91


In [17]:
#1-2
fifa.select(["Name","overall"]).orderBy(fifa["overall"].desc()).limit(5).toPandas()

,Name,overall
0,L. Messi,94
1,Cristiano Ronaldo,94
2,Neymar Jr,92
3,K. De Bruyne,91
4,De Gea,91


In [26]:
#1-3
fifa.select(["Name","overall"]).orderBy(["overall"],ascending=[0]).limit(5).toPandas()

,Name,overall
0,L. Messi,94
1,Cristiano Ronaldo,94
2,Neymar Jr,92
3,K. De Bruyne,91
4,De Gea,91


In [28]:
#2-1
fifa.select(["Name","potential"]).orderBy("potential",ascending=[0]).limit(5).toPandas()

,Name,potential
0,K. Mbappé,95
1,L. Messi,94
2,Cristiano Ronaldo,94
3,P. Dybala,94
4,Neymar Jr,93


In [33]:
fifa.select(["Name","overall","potential","Release Clause"]).orderBy(["overall","potential"],ascending=[0,0]).limit(5).toPandas()

,Name,overall,potential,Release Clause
0,L. Messi,94,94,€226.5M
1,Cristiano Ronaldo,94,94,€127.1M
2,Neymar Jr,92,93,€228.1M
3,De Gea,91,93,€138.6M
4,K. De Bruyne,91,92,€196.4M


#### count players who belong to a club of which its name start with FC


In [34]:
fifa.count()

18207

In [37]:
fifa.filter(fifa.Club.startswith("FC")).select(["Name","Age","Release Clause","Club"]).count()

1007

In [45]:
#Any Korean team in fifa?
#Let's find out distinct value of Nationality!

fifa.select(countDistinct(fifa.Nationality)).toPandas()

,count(DISTINCT Nationality)
0,164


In [59]:
#like function. 

fifa.select(fifa.Nationality).distinct().filter(fifa.Nationality.like("%Korea%")).toPandas()

,Nationality
0,Korea Republic
1,Korea DPR


In [60]:
#What Korean teams and players were registered? 
fifa.select(["Name","club"]).filter(fifa.Nationality.like("%Korea%")).toPandas()

,Name,club
0,H. Son,Tottenham Hotspur
1,Ki Sung Yueng,Newcastle United
2,Koo Ja Cheol,FC Augsburg
3,Kwon Chang Hoon,Dijon FCO
4,Lee Jae Sung,Holstein Kiel
...,...,...
334,Min Gyeong Min,Daegu FC
335,Lee Dong Hee,Jeju United FC
336,Kim Jong Jin,Gyeongnam FC
337,Hong Hyeon Seok,SpVgg Unterhaching


In [64]:
#who is the oldest and youngest player in the dataset?

print(fifa.orderBy(fifa.Age.desc()).select("Name","Age").show(5))

print(fifa.orderBy(fifa.Age.asc()).select("Name","Age").show(5))

+-------------+---+
|         Name|Age|
+-------------+---+
|     O. Pérez| 45|
|K. Pilkington| 44|
|    T. Warner| 44|
|  S. Narazaki| 42|
|    J. Villar| 41|
+-------------+---+
only showing top 5 rows

None
+------------+---+
|        Name|Age|
+------------+---+
|   B. Nygren| 16|
|H. Andersson| 16|
|    A. Doğan| 16|
|  C. Bassett| 16|
|    B. Mumba| 16|
+------------+---+
only showing top 5 rows

None


In [65]:
#L. Messi
#Cristiano Ronaldo

fifa.filter(fifa.Name.isin(["L. Messi","Cristiano Ronaldo"])).toPandas()

,_c0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,...,96,33,28,26,6,11,15,14,8,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,...,95,28,31,23,7,11,15,14,11,€127.1M


In [68]:
fifa.select(substring("Release Clause",0,1).alias("first_letter")).distinct().toPandas()

,first_letter
0,None
1,€


In [69]:
fifa.select("release clause").toPandas()

,release clause
0,€226.5M
1,€127.1M
2,€228.1M
3,€138.6M
4,€196.4M
...,...
18202,€143K
18203,€113K
18204,€165K
18205,€143K


In [82]:
fifa = fifa.withColumnRenamed("Release Clause","Release_Clause")

In [89]:
    
def udf_func(x):
    if "M" in x:
        x = x.replace('M',"*1000000")
    if "K" in x:
        x = x.replace("K","*1000")
    return (eval(x.replace("€","")))
custom_func = udf(lambda z: udf_func(z),StringType())

fifa.filter(fifa.Release_Clause.isNotNull()).select("name","club",custom_func(fifa["Release_Clause"]).alias("clause")).toPandas()


,name,club,clause
0,L. Messi,FC Barcelona,2.265E8
1,Cristiano Ronaldo,Juventus,1.271E8
2,Neymar Jr,Paris Saint-Germain,2.281E8
3,De Gea,Manchester United,1.386E8
4,K. De Bruyne,Manchester City,1.964E8
...,...,...,...
16638,J. Lundstram,Crewe Alexandra,143000
16639,N. Christoffersson,Trelleborgs FF,113000
16640,B. Worman,Cambridge United,165000
16641,D. Walker-Rice,Tranmere Rovers,143000


'tt'